# [2-1 CNN]

*KU LeeDongGyu*

### Note :

This code was used during the workshop at Kyungpook National University. <br>
Also, the content of this material is what I learned while moving Pycham to Jupyter.

Reference : TensorFlow 2.1 Quick Start Guide (by Holdroyd)

### Install

In [7]:
pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\82104\appdata\local\programs\python\python37\lib\site-packages (20.0.2)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install keras # 쓰지 말자.. tf.keras 쓰자..

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install theano
# keras에서 필요하다. 근데.. 그냥 tf.keras쓰자. keras만 쓰면 뭔가가 굉장히 안돌아감.(속도도 느림)

  Created wheel for theano: filename=Theano-1.0.4-cp37-none-any.whl size=2667193 sha256=d8511df41d4d8f70ae5d59a4319621f0f086aee50341b19a3aeab276d9456cb9
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\88\fb\be\483910ff7e9f703f30a10605ad7605f3316493875c86637014
Successfully built theano
Note: you may need to restart the kernel to use updated packages.


### Modules

In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# 현재 python2 버전을 쓰면서, 3버전의 함수가 필요할 경우, __future__ 모듈을 사용
# 즉, 나에게는 필요 없는 모듈이다. 자세한 내용은 아래 Reference를 참고.

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

## 1. MNIST
---

In [2]:
"""
참고교재: Advanced Deep Learning with Keras

예제 2-1-1: CNN classification on MNIST data set  
CNN을 구축하기 위해 "Functional API"를 사용함  
"""


# MNIST 데이터를 읽어들임
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
num_labels = len(np.unique(y_train)) # set 함수와 유사함.

In [4]:
# one-hot encoding
# keras.utils.to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [5]:
image_size = x_train.shape[1]
print(image_size)

28


In [6]:
x_train = np.reshape(x_train,[-1,image_size,image_size,1])
x_test = x_test.reshape([-1,image_size,image_size,1])

In [7]:
x_train.shape
# 데이터수, 가로, 세로, 깊이

(60000, 28, 28, 1)

In [8]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [9]:
# network parameters
input_shape = (image_size,image_size,1)
batch_size = 32
kernel_size = 3 # 3 x 3 필터
filters = 32 # 필터 32개
dropout = 0.3 # 0.7만 활성화

In [10]:
# CNN 만들기

inputs = Input(shape=input_shape)
# keras.layers.Input 임. class로 지정되어있음. 얘가 input층.

y = Conv2D(filters = filters,
          kernel_size = kernel_size,
          activation = 'relu')(inputs)
# keras.layers.Conv2D 임. class로 지정되어 있음. convolution층
y = MaxPooling2D()(y)
# keras.layers.MaxPooling2D 임. class로 지정되어 있음. pooling.

y = Conv2D(filters = filters, 
          kernel_size = kernel_size,
          activation = 'relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = Flatten()(y)
# keras.layers.Flatten 임. class로 지정되어 있음. fully connected.

y = Dropout(dropout)(y) 
# keras.layers.Dropout 임. class로 지정되어 있음. overfitting 방지.

outputs = Dense(num_labels, activation = 'softmax')(y)
# keras.layers.Dense 임. class로 지정되어 있음. classification.

In [11]:
model = Model(inputs = inputs , outputs = outputs)
# keras.models.Model 임. Class로 지정되어 있음. 모델로써 정의.
# keras.Model 도 똑같음.

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [13]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [15]:
model.fit(x_train,y_train,
         validation_data = (x_test,y_test),
         epochs=3,
         batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 45s 752us/sample - loss: 0.0701 - accuracy: 0.9781 - val_loss: 0.0313 - val_accuracy: 0.9889
Epoch 2/3
60000/60000 [==============================] - 46s 772us/sample - loss: 0.0534 - accuracy: 0.9834 - val_loss: 0.0373 - val_accuracy: 0.9879
Epoch 3/3
60000/60000 [==============================] - 46s 773us/sample - loss: 0.0447 - accuracy: 0.9858 - val_loss: 0.0272 - val_accuracy: 0.9912


In [16]:
score = model.evaluate(x_test,y_test,batch_size=batch_size)
print("\n Test accuracy: %.1f%%" % (100.0 * score[1]))

10000/10000 [==============================] - 2s 209us/sample - loss: 0.0272 - accuracy: 0.9912

 Test accuracy: 99.1%


## 2. fashion MNIST
---

In [17]:
"""
예제 2-1-2: CNN classification on fashion MNIST dataset

참고교재: Learn Tensorflow 2.0

https://raw.githubusercontent.com/Apress/learn-tensorflow-2/master/Chapter_2_3_and_4_code/cnn_fashion_mnist.py
"""

# Create class_names list object for mapping labels to names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [20]:
mnist_fashion = tf.keras.datasets.fashion_mnist

In [21]:
(training_images,training_labels),(test_images,test_labels) = mnist_fashion.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [22]:
#------------- Data Exploration ------------------#
# Shape of Training and Test Set
print('Training Dataset Shape: {}'.format(training_images.shape))
print('No. of Training Dataset Labels: {}'.format(len(training_labels)))
print('Test Dataset Shape: {}'.format(test_images.shape))
print('No. of Test Dataset Labels: {}'.format(len(test_labels)))


Training Dataset Shape: (60000, 28, 28)
No. of Training Dataset Labels: 60000
Test Dataset Shape: (10000, 28, 28)
No. of Test Dataset Labels: 10000


In [23]:
#------------- Data Preprocessing ------------------#
training_images = training_images / 255.0
test_images = test_images / 255.0

In [24]:
#------------- Reshaping the Training and Test dataset ------------------#
training_images = training_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

#각 값들을 float으로 바꾸고 나서, reshape을 해준다.

In [25]:
# Shape of Training and Test Set after applying reshape
print('Training Dataset Shape: {}'.format(training_images.shape))
print('No. of Training Dataset Labels: {}'.format(len(training_labels)))
print('Test Dataset Shape: {}'.format(test_images.shape))
print('No. of Test Dataset Labels: {}'.format(len(test_labels)))

Training Dataset Shape: (60000, 28, 28, 1)
No. of Training Dataset Labels: 60000
Test Dataset Shape: (10000, 28, 28, 1)
No. of Test Dataset Labels: 10000


In [27]:
#------------- Model Building ------------------#
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(50, (3, 3), activation='relu', input_shape=(28, 28, 1), name='Conv2D_layer'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling_2D'))
cnn_model.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model.add(tf.keras.layers.Dense(50, activation='relu', name='Hidden_layer'))
cnn_model.add(tf.keras.layers.Dense(10, activation='softmax', name='Output_layer'))


In [28]:
#------------- Check the details of different layers built in the CNN model ------------------#
cnn_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer (Conv2D)        (None, 26, 26, 50)        500       
_________________________________________________________________
Maxpooling_2D (MaxPooling2D) (None, 13, 13, 50)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 8450)              0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 50)                422550    
_________________________________________________________________
Output_layer (Dense)         (None, 10)                510       
Total params: 423,560
Trainable params: 423,560
Non-trainable params: 0
_________________________________________________________________


In [29]:
# tf.keras.utils.to_categorical 을 하지 않았음. -> sparse_categorical_crossentropy
#------------- Optimization function ------------------#
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [30]:
#------------- Model Training ------------------#
cnn_model.fit(training_images, training_labels, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 50s 838us/sample - loss: 0.4074 - accuracy: 0.8557
Epoch 2/3
60000/60000 [==============================] - 51s 854us/sample - loss: 0.2842 - accuracy: 0.8974
Epoch 3/3
60000/60000 [==============================] - 48s 807us/sample - loss: 0.2416 - accuracy: 0.9119


In [31]:
#------------- Model Evaluation ------------------#
# 1. Training Evaluation
training_loss, training_accuracy = cnn_model.evaluate(training_images, training_labels)
print('Training Accuracy {}'.format(round(float(training_accuracy), 2)))


60000/60000 [==============================] - 15s 252us/sample - loss: 0.2185 - accuracy: 0.9190
Training Accuracy 0.92


In [32]:
# 2. Test Evaluation
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels)
print('Test Accuracy {}'.format(round(float(test_accuracy), 2)))

10000/10000 [==============================] - 3s 286us/sample - loss: 0.2876 - accuracy: 0.8950
Test Accuracy 0.89


## 3. Reference
---

- [Module: future](http://www.hanbit.co.kr/channel/category/category_view.html?cms_code=CMS9324226566) 